In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from collections import defaultdict
import rdflib

In [3]:
file_path = '/home/zhang/hajarlamtaai/KGC/data_musee/wdump-3269.nt'

In [33]:
g = rdflib.Graph()

g.parse(file_path, format="nt")


query = """
SELECT ?type (COUNT(?entity) AS ?count) ?typeLabel WHERE {
  ?entity <http://www.wikidata.org/prop/direct/P31> ?type .
  OPTIONAL {
    ?type <http://www.w3.org/2000/01/rdfs-schema#label> ?typeLabel .
    FILTER (lang(?typeLabel) = "" || lang(?typeLabel) = "en")
  }
}
GROUP BY ?type ?typeLabel
"""


results = g.query(query)



Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#dateTime, Converter=<function parse_datetime at 0x300541a80>
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.11/site-packages/rdflib/term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/isodate/isodatetime.py", line 55, in parse_datetime
    tmpdate = parse_date(datestring)
              ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/isodate/isodates.py", line 203, in parse_date
    raise ISO8601Error('Unrecognised ISO 8601 date format: %r' % datestring)
isodate.isoerror.ISO8601Error: Unrecognised ISO 8601 date format: '-25000-01-01'


Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#dateTime, Converter=<function parse_datetime at 0x300541a80>
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.11/site-packages/rdflib/term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/isodate/isodatetime.py", line 55, in parse_datetime
    tmpdate = parse_date(datestring)
              ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/isodate/isodates.py", line 203, in parse_date
    raise ISO8601Error('Unrecognised ISO 8601 date format: %r' % datestring)
isodate.isoerror.ISO8601Error: Unrecognised ISO 8601 date format: '-0001-01-01'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#dateTime, Converter=<function parse_datetime at 0x300541a80>
Traceback (most recent call last):
  File

In [34]:
data = []
for row in results:
    type_uri = str(row.type)
    # print(help(row))
    count = row['count'].value
   
    type_label = str(row.typeLabel) if row.typeLabel else type_uri
    data.append((type_uri, type_label, count))

df = pd.DataFrame(data, columns=["Type URI", "Type Label", "Total Entities"])

# Display the DataFrame
print(df)

# Save the DataFrame to a CSV file
df.to_csv("types_and_counts.csv", index=False)

                                     Type URI   
0     http://www.wikidata.org/entity/Q3305213  \
1     http://www.wikidata.org/entity/Q3323788   
2    http://www.wikidata.org/entity/Q16905563   
3    http://www.wikidata.org/entity/Q11801536   
4     http://www.wikidata.org/entity/Q1278452   
..                                        ...   
298  http://www.wikidata.org/entity/Q56055312   
299    http://www.wikidata.org/entity/Q134560   
300    http://www.wikidata.org/entity/Q722604   
301  http://www.wikidata.org/entity/Q20437094   
302    http://www.wikidata.org/entity/Q751944   

                                   Type Label  Total Entities  
0     http://www.wikidata.org/entity/Q3305213           14491  
1     http://www.wikidata.org/entity/Q3323788               8  
2    http://www.wikidata.org/entity/Q16905563               3  
3    http://www.wikidata.org/entity/Q11801536               1  
4     http://www.wikidata.org/entity/Q1278452              14  
..                         

In [9]:
def parse_nt_file(file_path):
    data = {'subject': [], 'predicate': [], 'object': []}
    
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
    
            if line.endswith('.'):
                line = line[:-1].rstrip()
            
            parts = line.split(' ', 2)
            
            if len(parts) == 3:
                subject, predicate, object_ = parts
                
                subject = subject.strip('<>')
                predicate = predicate.strip('<>')
                object_ = object_.strip('<>')
                        
                data['subject'].append(subject)
                data['predicate'].append(predicate)
                data['object'].append(object_)
    
    df = pd.DataFrame(data)
    return df

In [10]:
df = parse_nt_file('/Users/grace/Desktop/KGC/data/data_musee/wdump-3269.nt')
df.to_csv('/Users/grace/Desktop/KGC/data/data_musee/raw_hrt.csv')




In [12]:

df.to_csv('raw_hrt.tsv', sep='\t', index=False)



In [5]:
def process_triplets(file_path):
    df = pd.read_csv(file_path, sep='\t')

    entity_to_index = {entity: idx for idx, entity in enumerate(pd.unique(df.values.ravel()))}

    df_mapped = df.applymap(lambda x: entity_to_index[x])

    return df_mapped

def analyze_connectivity(df_mapped):
    subject_counts = df_mapped.groupby('subject').size()
    object_counts = df_mapped.groupby('object').size()

    connectivity = {
        '1-1': 0,
        '1-n': 0,
        'n-1': 0,
        'n-n': 0
    }

    for subject, count in subject_counts.items():
        if count == 1:
            if subject in object_counts.index and object_counts[subject] == 1:
                connectivity['1-1'] += 1
            else:
                connectivity['1-n'] += 1
        else:
            if subject in object_counts.index and object_counts[subject] == 1:
                connectivity['n-1'] += 1
            else:
                connectivity['n-n'] += 1

    return connectivity

def main(file_path):
    df_mapped = process_triplets(file_path)
    connectivity = analyze_connectivity(df_mapped)
    print("Connectivity Analysis:")
    for key, value in connectivity.items():
        print(f"{key}: {value}")




In [21]:

triples_df = pd.read_csv(file_path, sep='\t', header=None, names=['head', 'relation', 'tail'])

entities = pd.concat([triples_df['head'], triples_df['tail']]).unique()
relations = triples_df['relation'].unique()


entity_to_index = {entity: idx for idx, entity in enumerate(entities)}
relation_to_index = {relation: idx for idx, relation in enumerate(relations)}


triples_df['head'] = triples_df['head'].map(entity_to_index)
triples_df['relation'] = triples_df['relation'].map(relation_to_index)
triples_df['tail'] = triples_df['tail'].map(entity_to_index)

entity_mapping_df = pd.DataFrame(list(entity_to_index.items()), columns=['entity', 'index'])
relation_mapping_df = pd.DataFrame(list(relation_to_index.items()), columns=['relation', 'index'])


entity_mapping_df.to_csv('entity_mapping.csv', index=False)
relation_mapping_df.to_csv('relation_mapping.csv', index=False)


triples_df.to_csv('mapped_triples.tsv', sep='\t', header=False, index=False)



In [22]:
mapped_triples_df = triples_df

triples_dict = {}

for _, row in mapped_triples_df.iterrows():
    head = row['head']
    relation_tail_pair = (row['relation'], row['tail'])
    
    if head not in triples_dict:
        triples_dict[head] = []
    
    triples_dict[head].append(relation_tail_pair)

In [23]:
triples_dict

{0: [(0, 28525)],
 1: [(1, 28526)],
 2: [(1, 28527)],
 3: [(1, 28526)],
 4: [(1, 28528)],
 5: [(1, 28526)],
 6: [(1, 28526)],
 7: [(1, 28527)],
 8: [(1, 28526)],
 9: [(1, 28527)],
 10: [(1, 28528)],
 11: [(1, 28528)],
 12: [(1, 28526)],
 13: [(1, 28526)],
 14: [(1, 28528)],
 15: [(1, 28527)],
 16: [(1, 28526)],
 17: [(1, 28527)],
 18: [(1, 28527)],
 19: [(1, 28526)],
 20: [(1, 28528)],
 21: [(1, 28528)],
 22: [(1, 28528)],
 23: [(1, 28526)],
 24: [(1, 28526)],
 25: [(1, 28528)],
 26: [(1, 28527)],
 27: [(1, 28526)],
 28: [(1, 28526)],
 29: [(1, 28527)],
 30: [(1, 28527)],
 31: [(1, 28528)],
 32: [(1, 28526)],
 33: [(1, 28526)],
 34: [(1, 28526)],
 35: [(1, 28527)],
 36: [(1, 28527)],
 37: [(1, 28526)],
 38: [(1, 28527)],
 39: [(1, 31),
  (2, 28529),
  (2, 28530),
  (3, 28531),
  (3, 28532),
  (4, 28533),
  (4, 28534),
  (4, 28535),
  (5, 28536),
  (6, 28537),
  (6, 28538),
  (7, 28539),
  (8, 28540),
  (9, 28541),
  (10, 28542),
  (11, 28543),
  (12, 28544),
  (13, 28545),
  (14, 28546

In [24]:
total_pairs = 0
total_heads = len(triples_dict)

# Iterate through the dictionary and count the pairs
for head, pairs in triples_dict.items():
    num_pairs = len(pairs)
    print(f"{head} has {num_pairs} pairs")
    total_pairs += num_pairs

# Calculate mean average
mean_average = total_pairs / total_heads

print(f"Mean average of pairs per head entity: {mean_average}")

0 has 1 pairs
1 has 1 pairs
2 has 1 pairs
3 has 1 pairs
4 has 1 pairs
5 has 1 pairs
6 has 1 pairs
7 has 1 pairs
8 has 1 pairs
9 has 1 pairs
10 has 1 pairs
11 has 1 pairs
12 has 1 pairs
13 has 1 pairs
14 has 1 pairs
15 has 1 pairs
16 has 1 pairs
17 has 1 pairs
18 has 1 pairs
19 has 1 pairs
20 has 1 pairs
21 has 1 pairs
22 has 1 pairs
23 has 1 pairs
24 has 1 pairs
25 has 1 pairs
26 has 1 pairs
27 has 1 pairs
28 has 1 pairs
29 has 1 pairs
30 has 1 pairs
31 has 1 pairs
32 has 1 pairs
33 has 1 pairs
34 has 1 pairs
35 has 1 pairs
36 has 1 pairs
37 has 1 pairs
38 has 1 pairs
39 has 63 pairs
40 has 3 pairs
41 has 3 pairs
42 has 3 pairs
43 has 3 pairs
44 has 3 pairs
45 has 3 pairs
46 has 3 pairs
47 has 3 pairs
48 has 3 pairs
49 has 3 pairs
50 has 3 pairs
51 has 3 pairs
52 has 2 pairs
53 has 3 pairs
54 has 1 pairs
55 has 2 pairs
56 has 3 pairs
57 has 1 pairs
58 has 2 pairs
59 has 3 pairs
60 has 1 pairs
61 has 2 pairs
62 has 3 pairs
63 has 1 pairs
64 has 2 pairs
65 has 3 pairs
66 has 1 pairs
67 h

In [6]:
#construct a subgraph limite to wikidata entities ..

def filter_triples(tsv_file, output):

    df = pd.read_csv(tsv_file, sep='\t', header=None, names=['subject', 'predicate', 'object'])
   
    filtered_df = df[df['subject'].str.startswith("http://www.wikidata.org/entity/")]
    

    filtered_triples = filtered_df.values.tolist()
    
    filtered_df.to_csv(output, sep='\t', index=False, header=False)
    
    return filtered_triples


In [9]:
# filter_triples(file_path, 'sub_graph.tsv')

In [32]:
def transductive_split(df, train_size=0.8, val_size=0.1, test_size=0.1, random_state=42):
    assert train_size + val_size + test_size == 1.0, "train, validation and test sizes must sum to 1.0"

    df = df.sample(frac=1, random_state=random_state).reset_index(drop=True)
    
    train_count = int(len(df) * train_size)
    val_count = int(len(df) * val_size)
    
    train_entities = set()
    train_relations = set()
    
    train_df = df.head(train_count)
    train_entities.update(train_df['subject'])
    train_entities.update(train_df['object'])
    train_relations.update(train_df['predicate'])
    
    remaining_df = df.iloc[train_count:]
    filtered_remaining_df = remaining_df[
        remaining_df['subject'].isin(train_entities) &
        remaining_df['object'].isin(train_entities) &
        remaining_df['predicate'].isin(train_relations)
    ]
    
    # Splitting the filtered remaining data
    val_df, test_df = train_test_split(
        filtered_remaining_df, 
        test_size=(test_size / (val_size + test_size)), 
        random_state=random_state
    )
    
    return train_df, val_df, test_df


In [33]:
df = pd.read_csv('/home/zhang/hajarlamtaai/KGC/data/data_musee/raw_hrt.tsv', sep='\t')

In [34]:
df.head()

,subject,predicate,object
0,http://wikiba.se/ontology#claim,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/2002/07/owl#ObjectProperty
1,http://www.w3.org/2004/02/skos/core#altLabel,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/2002/07/owl#DatatypeProperty
2,http://schema.org/about,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/2002/07/owl#ObjectProperty
3,http://wikiba.se/ontology#Property,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/2002/07/owl#Class
4,http://wikiba.se/ontology#statementProperty,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/2002/07/owl#ObjectProperty


In [35]:
print("DataFrame Index:", df.index)
print("DataFrame Columns:", df.columns)

DataFrame Index: RangeIndex(start=0, stop=484207, step=1)
DataFrame Columns: Index(['subject', 'predicate', 'object'], dtype='object')


In [36]:
train_df, val_df, test_df= transductive_split(df)

In [4]:
def parse_nt_file(file_path):
    graph = rdflib.Graph()
    graph.parse(file_path, format="nt")

    triples = []
    entities = set()
    relationships = set()
    
    for s, p, o in graph:
        triples.append((str(s), str(p), str(o)))
        entities.add(str(s))
        entities.add(str(o))
        relationships.add(str(p))
    
    return triples, entities, relationships

def calculate_cardinality(triples):
    subject_to_objects = defaultdict(set)
    object_to_subjects = defaultdict(set)
    
    for s, p, o in triples:
        subject_to_objects[(s, p)].add(o)
        object_to_subjects[(p, o)].add(s)
    
    one_to_one = 0
    one_to_many = 0
    many_to_one = 0
    many_to_many = 0
    
    total_relationships = len(subject_to_objects)
    
    for (s, p), objects in subject_to_objects.items():
        if len(objects) == 1:
            if len(object_to_subjects[(p, next(iter(objects)))]) == 1:
                one_to_one += 1
            else:
                one_to_many += 1
        else:
            many_to_many += 1
    
    for (p, o), subjects in object_to_subjects.items():
        if len(subjects) > 1:
            many_to_one += 1
    
    one_to_one_percentage = (one_to_one / total_relationships) * 100
    one_to_many_percentage = (one_to_many / total_relationships) * 100
    many_to_one_percentage = (many_to_one / total_relationships) * 100
    many_to_many_percentage = (many_to_many / total_relationships) * 100
    
    return one_to_one_percentage, one_to_many_percentage, many_to_one_percentage, many_to_many_percentage

def main(file_path):
    triples, entities, relationships = parse_nt_file(file_path)
    one_to_one_percentage, one_to_many_percentage, many_to_one_percentage, many_to_many_percentage = calculate_cardinality(triples)
    
    print(f"# Triples: {len(triples)}")
    print(f"# Entities: {len(entities)}")
    print(f"# Relationships: {len(relationships)}")
    print(f"1-1: {one_to_one_percentage:.2f}%")
    print(f"1-n: {one_to_many_percentage:.2f}%")
    print(f"n-1: {many_to_one_percentage:.2f}%")
    print(f"n-n: {many_to_many_percentage:.2f}%")

file_path = 'data/data_musee/wdump-3269.nt'
main(file_path)


Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#dateTime, Converter=<function parse_datetime at 0x108953920>
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.11/site-packages/rdflib/term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/isodate/isodatetime.py", line 55, in parse_datetime
    tmpdate = parse_date(datestring)
              ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/isodate/isodates.py", line 203, in parse_date
    raise ISO8601Error('Unrecognised ISO 8601 date format: %r' % datestring)
isodate.isoerror.ISO8601Error: Unrecognised ISO 8601 date format: '-25000-01-01'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#dateTime, Converter=<function parse_datetime at 0x108953920>
Traceback (most recent call last):
  Fil

# Triples: 483938
# Entities: 175822
# Relationships: 370
1-1: 28.06%
1-n: 54.66%
n-1: 5.29%
n-n: 17.28%
